In [ ]:
# Imports
%load_ext autoreload
%autoreload 2

import json
from dwitracts.main import DwiTracts
import dwitracts.plot as plot

In [ ]:
# Initialize a DwiTract project
config_file = 'project/config_tracts_dmn.json'
params = json.load(config_file)

my_dwi = DwiTracts( params )

assert my_dwi.initialize()


In [ ]:
# Compute bidirectional average distributions for each tract (binary and minimum approaches)

assert my_dwi.compute_bidirectional_averages( verbose=True, clobber=True, debug=True )


In [ ]:
# Compute tract distances (A->B and B->A) for each tract

assert my_dwi.compute_tract_distances( verbose=True, clobber=True, debug=True )


In [ ]:
# Generate core polylines, gaussian uncertainty fields, and unidirectional tract estimates 

assert my_dwi.estimate_unidirectional_tracts( verbose=True, clobber=True, debug=True )


In [ ]:
# Generate bidirectional tract estimates 

assert my_dwi.estimate_bidirectional_tracts( verbose=True, clobber=True )


In [ ]:
# Create a pass/fail Pajek graph

assert my_dwi.tracts_to_pajek( verbose=True, clobber=True )


In [ ]:
# Compute voxel-wise average streamline orientations for each tract

assert my_dwi.compute_average_orientations( verbose=True, clobber=True )


In [ ]:
# Compute tract-specific anisotropy

assert my_dwi.compute_tsa( verbose=True, clobber=True )


In [ ]:
# Generate average TSA images for each tract

assert my_dwi.generate_mean_tsa_images( verbose=True )


In [ ]:
# Plot TSA histograms
params_plot = {}
params_plot['axis_font'] = 18
params_plot['ticklabel_font'] = 12
params_plot['title_font'] = 18
params_plot['show_labels'] = True
params_plot['show_title'] = False
params_plot['dimensions_tracts'] = (50,40)
params_plot['dimensions_all'] = (50,40)
params_plot['dpi_tracts'] = 150
params_plot['dpi_all'] = 300
params_plot['num_bins'] = 20
params_plot['kde'] = True
params_plot['stat'] = 'density'
params_plot['xlim'] = [-0.5, 1]
params_plot['xticks'] = [-0.5, 0.0, 0.5, 1.0]
params_plot['color'] = '#2b3ad1'  # blue

tract_names = None

for threshold in [0.1,0.5]:
    stats = plot.plot_tsa_histograms( params_plot, my_dwi, tract_names=tract_names, threshold=threshold, \
                                      verbose=True, clobber=True )

    # Save TSA stats to CSV file
    stats.to_csv('{0}/stats_tsa_{1:02d}.csv'.format(my_dwi.tracts_dir, threshold*100))
